<a href="https://colab.research.google.com/github/PeteJoh/fpl-ai/blob/master/weekly_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import pandas as pd

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:



def get_team_name(id):
    general_information = requests.get(
        "https://fantasy.premierleague.com/api/bootstrap-static/"
    ).json()
    teams = general_information["teams"]
    for team in teams:
        if team["id"] == id:
            return team["name"]


def get_player_fixture_info(id):
    player_info = requests.get(
        f"https://fantasy.premierleague.com/api/element-summary/{id}/"
    ).json()
    fixture = player_info["fixtures"][0]
    home_team = get_team_name(fixture["team_h"])
    away_team = get_team_name(fixture["team_a"])
    kickoff_time = fixture["kickoff_time"]
    is_home = fixture["is_home"]
    return home_team, away_team, kickoff_time, is_home


def get_gameweek_result(id, gameweek):
    "returns a dictionary of the statistics from the gameweek"
    gameweek_results = requests.get(
        f"https://fantasy.premierleague.com/api/element-summary/{id}/"
    ).json()
    return gameweek_results["history"][gameweek - 1]


def get_player_position(element_type):
    general_information = requests.get(
        "https://fantasy.premierleague.com/api/bootstrap-static/"
    ).json()
    positions = general_information["element_types"]
    for position in positions:
        if position["id"] == element_type:
            return position["singular_name_short"]


general_info = requests.get(
    f"https://fantasy.premierleague.com/api/bootstrap-static/"
).json()
player_infos = general_info["elements"]
gameweek = 1

"""gameweek_fixture = pd.read_csv(f"datasets/gw4.csv")
gameweek_fixture = pd.read_csv(
       f"https://raw.githubusercontent.com/vaastav/Fantasy-Premier-League/master/data/2023-24/gws/gw4.csv"
    )

print(gameweek_fixture.columns)

def get_fixture(matchday, name):
    # load week's fixture
    global gameweek_fixture
    gameweek_fixture_ = gameweek_fixture[gameweek_fixture["name"] == name]
    #print(gameweek_fixture_["kickoff_time"].iloc[0])
    kickoff_time=gameweek_fixture_["kickoff_time"].iloc[0]
    team=gameweek_fixture_["team"].iloc[0]
    is_home=gameweek_fixture_["was_home"].iloc[0]
    opponent_team=get_team_name(gameweek_fixture_["opponent_team"].iloc[0])
    if is_home:
        home_team,away_team=team,opponent_team
    else:
        home_team,away_team=opponent_team,team

    return (
        home_team,
        away_team,
        kickoff_time,
        is_home
    )"""


gameweek_fixture = pd.read_csv(f"/content/gdrive/MyDrive/fplpredict/datasets/2024-25/fixtures/GW{gameweek}.csv")


def get_fixture(matchday, name):
    # load week's fixture
    global gameweek_fixture
    gameweek_fixture_ = gameweek_fixture[gameweek_fixture["name"] == name]
    return (
        gameweek_fixture_["home_team"].iloc[0],
        gameweek_fixture_["away_team"].iloc[0],
        gameweek_fixture_["kickoff_time"].iloc[0],
        gameweek_fixture_["is_home"].iloc[0],
    )

# week 6 results
all_players = []
for player_info in player_infos:
    try:
        #print(all_players)
        id = player_info["id"]
        print(id)
        name = player_info["first_name"] + " " + player_info["second_name"]
        team_id = player_info["team"]
        cost = player_info["now_cost"]
        element_type = player_info["element_type"]
        my_team = get_team_name(team_id)
        position = get_player_position(element_type)
        gameweek_result = get_gameweek_result(id, gameweek)
        home_team, away_team, kickoff_time, is_home = get_fixture(gameweek, name)
        #print(gameweek_result)
        gameweek_result["id"] = id
        gameweek_result["name"] = name
        gameweek_result["cost"] = cost
        gameweek_result["position"] = position
        gameweek_result["home_team"] = home_team
        gameweek_result["away_team"] = away_team
        gameweek_result["team_x"] = my_team
        gameweek_result["opponent_team"] = get_team_name(
            gameweek_result["opponent_team"]
        )
        all_players.append(gameweek_result)

    except IndexError:
        continue

# get x week result
print(all_players)
df = pd.DataFrame(all_players)
print(df)

FileNotFoundError: [Errno 2] No such file or directory: '/content/gdrive/MyDrive/fplpredict/datasets/2024-25/fixtures/GW1.csv'

In [ ]:
df

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,selected,transfers_in,transfers_out,id,name,cost,position,home_team,away_team,team_x
0,1,84,Chelsea,0,False,2023-10-21T16:30:00Z,2,2,9,0,...,26039,0,1453,1,Folarin Balogun,44,FWD,Chelsea,Arsenal,Arsenal
1,2,84,Chelsea,0,False,2023-10-21T16:30:00Z,2,2,9,0,...,42211,2986,1814,2,Cédric Alves Soares,39,DEF,Chelsea,Arsenal,Arsenal
2,3,84,Chelsea,0,False,2023-10-21T16:30:00Z,2,2,9,0,...,8786,1006,682,3,Mohamed Elneny,44,MID,Chelsea,Arsenal,Arsenal
3,4,84,Chelsea,0,False,2023-10-21T16:30:00Z,2,2,9,0,...,12793,313,1175,4,Fábio Ferreira Vieira,54,MID,Chelsea,Arsenal,Arsenal
4,5,84,Chelsea,1,False,2023-10-21T16:30:00Z,2,2,9,90,...,1303504,106197,44609,5,Gabriel dos Santos Magalhães,47,DEF,Chelsea,Arsenal,Arsenal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
653,572,82,Bournemouth,0,False,2023-10-21T14:00:00Z,1,2,9,0,...,39501,2854,6654,572,Nélson Cabral Semedo,45,DEF,Bournemouth,Wolves,Wolves
654,573,82,Bournemouth,2,False,2023-10-21T14:00:00Z,1,2,9,74,...,8879,1147,377,573,Toti António Gomes,44,DEF,Bournemouth,Wolves,Wolves
655,574,82,Bournemouth,1,False,2023-10-21T14:00:00Z,1,2,9,90,...,47642,2132,3041,574,Boubacar Traoré,44,MID,Bournemouth,Wolves,Wolves
656,590,82,Bournemouth,7,False,2023-10-21T14:00:00Z,1,2,9,81,...,75828,15978,6998,590,Matheus Santos Carneiro Da Cunha,55,FWD,Bournemouth,Wolves,Wolves


In [ ]:
df[df["team_x"]=="Man City"]

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,selected,transfers_in,transfers_out,id,name,cost,position,home_team,away_team,team_x
392,341,86,Brighton,-1,True,2023-10-21T14:00:00Z,2,1,9,90,...,720853,10173,59847,341,Manuel Akanji,50,DEF,Man City,Brighton,Man City
393,342,86,Brighton,1,True,2023-10-21T14:00:00Z,2,1,9,1,...,628213,14983,31082,342,Nathan Aké,51,DEF,Man City,Brighton,Man City
394,343,86,Brighton,9,True,2023-10-21T14:00:00Z,2,1,9,90,...,3332646,214415,163939,343,Julián Álvarez,71,FWD,Man City,Brighton,Man City
395,344,86,Brighton,2,True,2023-10-21T14:00:00Z,2,1,9,90,...,135308,11121,5042,344,Bernardo Veiga de Carvalho e Silva,63,MID,Man City,Brighton,Man City
396,345,86,Brighton,0,True,2023-10-21T14:00:00Z,2,1,9,0,...,5477,738,520,345,Oscar Bobb,44,MID,Man City,Brighton,Man City
397,346,86,Brighton,0,True,2023-10-21T14:00:00Z,2,1,9,0,...,116480,0,3049,346,João Cancelo,58,DEF,Man City,Brighton,Man City
398,347,86,Brighton,0,True,2023-10-21T14:00:00Z,2,1,9,0,...,9098,435,315,347,Scott Carson,39,GKP,Man City,Brighton,Man City
399,348,86,Brighton,0,True,2023-10-21T14:00:00Z,2,1,9,0,...,37,0,1,348,Shea Charles,45,MID,Man City,Brighton,Man City
400,349,86,Brighton,0,True,2023-10-21T14:00:00Z,2,1,9,0,...,168263,492,3452,349,Kevin De Bruyne,102,MID,Man City,Brighton,Man City
401,350,86,Brighton,1,True,2023-10-21T14:00:00Z,2,1,9,15,...,824475,19195,32912,350,Rúben Gato Alves Dias,55,DEF,Man City,Brighton,Man City


In [ ]:
df[df["team_x"]=="Wolves"]

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,selected,transfers_in,transfers_out,id,name,cost,position,home_team,away_team,team_x
162,156,83,Brentford,0,False,2023-10-21T14:00:00Z,3,0,9,0,...,11555,369,650,156,Enock Agyei,44,MID,Brentford,Burnley,Burnley
163,157,83,Brentford,1,False,2023-10-21T14:00:00Z,3,0,9,90,...,78714,3832,9270,157,Ameen Al-Dakhil,40,DEF,Brentford,Burnley,Burnley
164,158,83,Brentford,0,False,2023-10-21T14:00:00Z,3,0,9,0,...,1661,0,98,158,Samuel Bastien,49,MID,Brentford,Burnley,Burnley
165,159,83,Brentford,0,False,2023-10-21T14:00:00Z,3,0,9,0,...,7563,40,616,159,Manuel Benson Hedilazio,52,MID,Brentford,Burnley,Burnley
166,160,83,Brentford,0,False,2023-10-21T14:00:00Z,3,0,9,0,...,211111,655,41754,160,Jordan Beyer,40,DEF,Brentford,Burnley,Burnley
167,161,83,Brentford,1,False,2023-10-21T14:00:00Z,3,0,9,24,...,18627,1190,2269,161,Josh Brownhill,48,MID,Brentford,Burnley,Burnley
168,162,83,Brentford,0,False,2023-10-21T14:00:00Z,3,0,9,0,...,2060,192,118,162,Darko Churlinov,44,MID,Brentford,Burnley,Burnley
169,163,83,Brentford,0,False,2023-10-21T14:00:00Z,3,0,9,0,...,2297,53,86,163,Jack Cork,48,MID,Brentford,Burnley,Burnley
170,164,83,Brentford,0,False,2023-10-21T14:00:00Z,3,0,9,0,...,1190,0,49,164,Dara Costelloe,45,MID,Brentford,Burnley,Burnley
171,165,83,Brentford,1,False,2023-10-21T14:00:00Z,3,0,9,90,...,8697,318,985,165,Josh Cullen,50,MID,Brentford,Burnley,Burnley


In [ ]:
#save to folder
df.to_csv(f"/content/gdrive/MyDrive/fplpredict/datasets/2024-25/results/GW{gameweek}.csv")